In [ ]:
window = 5
data['AAPL_roll_mean'] = data['AAPL_log'].rolling(window).mean()
data['AAPL_roll_std'] = data['AAPL_log'].rolling(window).std()

# Define exit signal: next day AAPL_log < -0.01 (example)
data['exit_signal'] = (data['AAPL_log'].shift(-1) < -0.01).astype(int)

data = data.dropna()

# Features and target
features = data[['AAPL_log', 'AAPL_roll_mean', 'AAPL_roll_std']]	
target = data['exit_signal']

# Train-test split
split = int(len(data) * 0.8)
X_train, X_test = features.iloc[:split], features.iloc[split:]
y_train, y_test = target.iloc[:split], target.iloc[split:]

# Train RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict exit on full data
exit_preds = pd.DataFrame({
    'AAPL': model.predict(features).astype(bool)
}, index=features.index)

# Simple entry: price > 20-day SMA on aligned close prices
close = pd.DataFrame({
    'AAPL': aapl['close'].loc[data.index]
})
entries = close > close.rolling(20).mean()
entries = entries.fillna(False)


# Use test data only
test_close = close.iloc[split:]
test_entries = entries.iloc[split:]
test_exit_preds = exit_preds.iloc[split:]

# Backtest on test data
pf = vbt.Portfolio.from_signals(
    test_close,       
    test_entries, 
    test_exit_preds,
    init_cash=100000,
    freq='1D',
    fees=0.001
)

print(pf.stats())

In [ ]:
from scipy.stats import rankdata

pobs = resids.apply(lambda x: rankdata(x) / (len(x) + 1), axis=0)
sns.pairplot(pobs)
plt.suptitle("Pseudo-observations from GARCH residuals", y=1.02)
plt.show()

In [ ]:
from scipy.stats import norm

asset = "AAPL_log"  # replace with your actual asset name

# 1. Transform residuals to uniform marginals for just one asset
resid_uniform = norm.cdf(resids[asset])

# 2. Compute copula likelihood
# Transform all residuals to uniform
resids_uniform = pd.DataFrame(norm.cdf(resids), columns=resids.columns, index=resids.index)
#print(resids_uniform)

# Joint likelihoods (for multivariate analysis)
likelihoods = copula.probability_density(resids_uniform)


# 3. Threshold for defining extreme days
threshold = np.percentile(likelihoods, 15) # Tweak percenmtile as needed
extreme_days = likelihoods < threshold

# 4. Create signal series
signals = pd.Series(index=resids.index, dtype=object)
extreme_resid = resids[asset][extreme_days]
signals.loc[extreme_resid.index] = np.where(extreme_resid < 0, 'buy', 'sell')
signals.fillna('hold', inplace=True)

# 5. Create entry/exit boolean Series
entries = signals == 'buy'
exits = signals == 'sell'


In [ ]:
corr = pd.DataFrame(samples, columns=resids.columns).corr()
plt.figure(facecolor="#1A1A1A")

sns.heatmap(
    corr,
    annot=True,
    cmap="Oranges",
    cbar_kws={'label': 'Correlation'},
    annot_kws={'color': '#000000'},
    linewidths=0.5,
    linecolor="#494949"
)

plt.title("Correlation heatmap of copula samples", color='white')
plt.gca().set_facecolor("#292929")  # panel background
plt.xticks(color='white')
plt.yticks(color='white')
plt.show()

Trying to trade all at once

In [ ]:
signals_dict = {}
for asset in stock_params.keys():
    signals_dict[asset] = pd.Series('hold', index=resids.index)

for date in resids.index:
    # Extract residuals for all assets on that date
    day_resids = resids.loc[date, [f"{a}_log" for a in stock_params.keys()]]
    
    # Find asset with max absolute residual
    max_asset_idx = day_resids.abs().idxmax()
    max_asset = max_asset_idx.replace('_log', '')
    max_resid = day_resids[max_asset_idx]
    
    # Set signal for that asset based on sign
    if max_resid < 0:
        signals_dict[max_asset].loc[date] = 'buy'
    else:
        signals_dict[max_asset].loc[date] = 'sell'

In [ ]:
from vectorbt.portfolio.enums import SizeType, Direction

price_dfs = {}
for ticker in tickers:
    df = pd.read_csv(f"case_data/{ticker}.csv", index_col=1, parse_dates=True)
    price_dfs[ticker] = df['close']

# Combine closes into one DataFrame
price_df = pd.concat([price_dfs[ticker] for ticker in tickers], axis=1)
price_df.columns = tickers

# Align with your cleaned data index (same as signals index)
price_df = price_df.reindex(data_clean.index)

# Convert signals to numeric weights (buy=1, sell=0, hold=0)
weights = pd.DataFrame(index=resids.index, columns=signals_dict.keys())
for asset, sig in signals_dict.items():
    weights[asset] = sig.map({'buy': 1, 'sell': 0, 'hold': 0}).fillna(0)

# Normalize weights to sum to 1 per day (optional, only if you want full allocation every day)
weights = weights.div(weights.sum(axis=1).replace(0, np.nan), axis=0).fillna(0)

# Load close prices for all assets as DataFrame aligned with weights
price_df = pd.concat([price_dfs[asset] for asset in weights.columns], axis=1)
price_df.columns = weights.columns
price_df = price_df.reindex(weights.index)

pf = vbt.Portfolio.from_orders(
    close=price_df,
    size=weights,
    size_type=SizeType.TargetPercent,
    direction=Direction.LongOnly,
    call_seq='auto',
    group_by=True,
    cash_sharing=True,
    init_cash=100000,
    fees=0.001,
    slippage=0.0002,
    freq='1D'
)

print(pf.stats())
pf.plot(template='plotly_dark').show()

In [ ]:
from scipy.stats import norm

asset = "AAPL_log"  # replace with your actual asset name

# 1. Transform residuals to uniform marginals for just one asset
resid_uniform = norm.cdf(resids[asset])

# 2. Compute copula likelihood
# Transform all residuals to uniform
resids_uniform = pd.DataFrame(norm.cdf(resids), columns=resids.columns, index=resids.index)
#print(resids_uniform)

# Joint likelihoods (for multivariate analysis)
likelihoods = copula.probability_density(resids_uniform)


# Compute lower and upper thresholds
lower_thresh = np.percentile(likelihoods, 0)
upper_thresh = np.percentile(likelihoods, 20)

# Define extreme days between 5th and 25th percentile
extreme_days = (likelihoods >= lower_thresh) & (likelihoods < upper_thresh)

# Create signal series
signals = pd.Series(index=resids.index, dtype=object)
extreme_resid = resids[asset][extreme_days]
signals.loc[extreme_resid.index] = np.where(extreme_resid < 0, 'buy', 'sell')
signals.fillna('hold', inplace=True)

# Entry/exit logic
entries = signals == 'buy'
exits = signals == 'sell'

In [ ]:
# Load close prices for the relevant ticker
price_df = pd.read_csv("case_data/AAPL.csv", index_col=1, parse_dates=True)['close']

# Align with entries index
price_df = price_df.loc[entries.index]

pf = vbt.Portfolio.from_signals(
    price_df,
    entries,
    exits,
    call_seq='auto',                # sell first, then buy
    init_cash=100000,               # starting capital
    fees=0.001,                     # 0.1% round‐trip
    slippage=0.0002,
    freq='1D'
)

print(pf.stats())
pf.plot(template='plotly_dark').show()


# Plot drawdowns
pf.drawdowns.plot(template='plotly_dark').show()

# print(pf.stats()["Sharpe Ratio"]) # isolates sharpe ratio


In [ ]:
corr = pd.DataFrame(samples, columns=resids.columns).corr()
plt.figure(facecolor="#1A1A1A")

sns.heatmap(
    corr,
    annot=True,
    cmap="Oranges",
    cbar_kws={'label': 'Correlation'},
    annot_kws={'color': '#000000'},
    linewidths=0.5,
    linecolor="#494949"
)

plt.title("Correlation heatmap of copula samples", color='white')
plt.gca().set_facecolor("#292929")  # panel background
plt.xticks(color='white')
plt.yticks(color='white')
plt.show()

In [ ]:
portfolios = {}

for asset in tickers:
    price_df = pd.read_csv(f"case_data/{asset}.csv", index_col=1, parse_dates=True)['close']
    
    entries = signals_dict[asset] == 'buy'
    exits = signals_dict[asset] == 'sell'
    
    # Align price with signal index
    price_df = price_df.reindex(entries.index)
    
    pf = vbt.Portfolio.from_signals(
        price_df,
        entries,
        exits,
        call_seq='auto',
        init_cash=100000,
        fees=0.001,
        slippage=0.0002,
        freq='1D'
    )
    
    portfolios[asset] = pf
    print(f"{asset} stats:\n", pf.stats())
    pf.plot(template='plotly_dark').show()
    pf.drawdowns.plot(template='plotly_dark').show()


In [ ]:

# Transform to uniform space (copula space)
samples_uniform = pd.DataFrame(norm.cdf(samples), columns=resids.columns)

# Plot in copula (uniform) space
sns.set_palette(["#ff6600"])
g = sns.pairplot(samples_uniform, plot_kws={'edgecolor': '#000000'})

# Dark background
for ax in g.axes.flatten():
    if ax:
        ax.set_facecolor("#292929")
        ax.tick_params(colors='white')
        ax.spines[:].set_color('white')

plt.suptitle("Copula Samples in Uniform Space [0, 1]", y=1.02, color='white')
plt.gcf().set_facecolor("#1A1A1A")
plt.show()

In [ ]:
from copulas.multivariate import Multivariate

copula = GaussianMultivariate()
copula.fit(resids)
# print(resids.head()) 

# You can sample from the fitted copula or inspect parameters
samples = copula.sample(len(resids))
print("Fitted copula parameters:", copula.to_dict())

# Dark-mode Orange
sns.set_style("darkgrid", {
    'figure.facecolor': "#1A1A1A",       # background for main plot
    'grid.color': "#494949",             # grid
    'axes.edgecolor': "#ffffff",         # white border 
    'axes.labelcolor': '#ffffff',        # white axis label
    'text.color': '#ffffff',             # white text
    'patch.edgecolor': "#000000",        # edges of histogram bars
})

sns.set_palette(["#ff6600"])

# Visualize the samples
g = sns.pairplot(pd.DataFrame(samples, columns=resids.columns),
                 plot_kws={'edgecolor': '#000000'})



# Apply dark background to each subplot
for ax in g.axes.flatten():
    if ax:
        ax.set_facecolor("#292929")

# Title styling
plt.suptitle("Samples from fitted copula", y=1.02, color='white')
plt.gcf().set_facecolor("#1A1A1A")

plt.show()